In [0]:
storageAccountName = 'equitymarketstorage'
storageAccountAccessKey = '32Q+pSDygN/vBS/+2OPfjXJrW9+T4gogsOJY3mEob/GON/Hls27FX1ZaxbBxEjsmA+/+XJn1AXkFj2dUSrOeWA=='
blobContainerName = 'firstcontainer'


if not any(mount.mountPoint == '/mnt/FileStore/MountFolder/' for mount in dbutils.fs.mounts()):
  try:
    dbutils.fs.mount(
    source = "wasbs://{}@{}.blob.core.windows.net".format(blobContainerName, storageAccountName),
    mount_point = "/mnt/FileStore/MountFolder/",
    extra_configs = {'fs.azure.account.key.' + storageAccountName + '.blob.core.windows.net': storageAccountAccessKey}
  )
  except Exception as e:
    print("already mounted. Try to unmount first")
    
display(dbutils.fs.ls("dbfs:/mnt/FileStore/MountFolder/")) 

path,name,size
dbfs:/mnt/FileStore/MountFolder/data/,data/,0
dbfs:/mnt/FileStore/MountFolder/spark_data/,spark_data/,0


In [0]:
from pyspark.sql import SparkSession
raw_rdd=spark.read.csv("dbfs:/mnt/FileStore/MountFolder/data/car_data.csv",schema='''
                                                incident_id INT,
                                                incident_type STRING,
                                                vin_number STRING,
                                                make STRING,
                                                model STRING,
                                                year date,
                                                incident_date date,
                                                description STRING
                                            ''')


In [0]:
infoDF = raw_rdd.filter('incident_type == "I"').select('incident_type','vin_number','make','year')
accidentDF = raw_rdd.filter('incident_type == "A"').select('incident_type','vin_number','make','year')
enhanceDF=infoDF.join(accidentDF, 'vin_number').select(infoDF.make, infoDF.year)
finalDF=enhanceDF.groupBy("make","year").count()


In [0]:
display(finalDF)

make,year,count
Mercedes,2016-01-01,1
Nissan,2003-01-01,1
Mercedes,2015-01-01,2
